In [1]:
%load_ext autoreload
%autoreload 2
from ASD_Circuits import *
from venn import venn

HGNC, ENSID2Entrez, GeneSymbol2Entrez, Entrez2Symbol, allen_mouse_genes = LoadGeneINFO()
ExpMat, ExpZscoreMat, ExpMatNorm, ExpZscoreMatNorm = LoadExpressionMatrices()
ExpMat = pd.read_csv("dat/Jon_data/energy-exp_level.csv", index_col="ROW", low_memory=False)
gnomad_cons = pd.read_csv("../dat/genes/gnomad.v2.1.1.lof_metrics.by_gene.txt",
                          delimiter="\t", index_col="gene", low_memory=False)

In [3]:
SparkMetaBias = pd.read_csv("dat/Jon_data/Spark_Final_ExpBias.csv")
SparkMetaBias.columns = ["STR", "EFFECT", "Rank", "NGene"]
SparkMetaBias = SparkMetaBias.set_index("STR")

# ASD bootstraped biases
biases_asd_boot = []
asd_boot_dir = "dat/asd.boot.spec/"
for file in os.listdir(asd_boot_dir):
    df = pd.read_csv(asd_boot_dir+file, index_col="STR")
    biases_asd_boot.append(df)

In [6]:
adj_mat = pd.read_csv(ConnFil, index_col=0)
g = LoadConnectome2(adj_mat) # Load Connectiome
EdgeWeightsDict = EdgeDict(g, keyon="label")
str2reg = STR2Region()
str2reg_df = pd.read_csv(MajorBrainDivisions, delimiter="\t")

In [37]:
def CohesivenessProfile(BiasDF, g=g, EdgeWeightsDict={}):
    EdgeWeightsDict = {}
    topNs = list(range(200, 10, -10))
    ASD_Conn_Z, ASD_Cohe_Z, ASD_Conn_P, ASD_Cohe_P, ASD_Conn_E, ASD_Cohe_E = [],[],[],[],[],[]
    for topN in topNs:
        Permuted_ASD_cohe = []
        Permuted_ASD_conn = []
        Permuted_Sib_cohe = []
        Permuted_Sib_conn = []
        for i in range(100):
            adj_mat_perm = pd.read_csv("dat/permut_connectome/{}.csv".format(i), index_col=0)
            g_perm = LoadConnectome2(adj_mat_perm)
            asd_cohe, asd_conn = ScoreSTRSet(g_perm, BiasDF.head(topN).index.values, EdgeWeightsDict)
            Permuted_ASD_cohe.append(asd_cohe)
            Permuted_ASD_conn.append(asd_conn)
        asd_cohe, asd_conn = ScoreSTRSet(g, BiasDF.head(topN).index.values, EdgeWeightsDict)

        asd_z_conn, asd_p_conn = GetPermutationP(Permuted_ASD_conn, asd_conn)
        asd_z_cohe, asd_p_cohe = GetPermutationP(Permuted_ASD_cohe, asd_cohe)
        effect_conn = asd_conn/np.mean(Permuted_ASD_conn)
        effect_cohe = asd_cohe/np.mean(Permuted_ASD_cohe)
        ASD_Conn_Z.append(asd_z_conn); ASD_Cohe_Z.append(asd_z_cohe)
        ASD_Conn_P.append(asd_p_conn); ASD_Cohe_P.append(asd_p_cohe)
        ASD_Conn_E.append(effect_conn); ASD_Cohe_E.append(effect_cohe)
    return ASD_Cohe_E, ASD_Cohe_P

In [38]:
SparkE, SparkP = CohesivenessProfile(SparkMetaBias)

In [39]:
toplotbootE = []
toplotbootP = []
for i in range(100):
    BootE, BootP = CohesivenessProfile(biases_asd_boot[i])
    toplotbootE.append(BootE)
    toplotbootP.append(BootP)

In [40]:
toplotbootE = np.array(toplotbootE)
toplotbootE.mean(axis=0).shape

(19,)

In [41]:
topNs = list(range(200, 10, -10))
def addline(TopNs, Eff, ax, color="grey", ls="dashed"):
    ax.plot(TopNs, Eff, color=color, ls="solid", lw=3)

In [42]:
%matplotlib inline
mpl.rcParams.update(mpl.rcParamsDefault)
fig, ax = plt.subplots(dpi=720, figsize=(16,8))
plt.style.use('seaborn-talk')
matplotlib.rcParams.update({'font.size': 32})

addline(topNs, SparkE, ax, "blue")

ax.hlines(xmin=min(topNs), xmax=max(topNs), y=1, ls="--", color="grey")
ax.grid(True)

addline(topNs, toplotbootE.mean(axis=0), ax, "grey")
#for i in range(100):
#    addline(topNs, toplotbootE[i], ax, "grey")

#ax.plot(np.arange(0), np.arange(0), label="ASD Probands", ls="solid", color="blue")
#ax.plot(np.arange(0), np.arange(0), label="Siblings", ls="solid", color="orange")
#ax.plot(np.arange(0), np.arange(0), label='Significant', ls="solid", color="grey")
#ax.plot(np.arange(0), np.arange(0), label='Not Significant', ls="dashed", color="grey")
        ## P_perm < %.3e'%(0.05/213)

#ax.legend(fontsize=24, loc="lower right")
ax.set_ylabel("Cohesiveness Effect Size",fontsize=28)
ax.set_xlabel("Number of Most Biased Structures",fontsize=28)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
#plt.legend(['ASD Probands', 'Siblings', 
#            'P_perm < %.3e'%(0.05/213), 'P_perm > %.3e'%(0.05/213)], 
#          fontsize=24, loc="lower right")
plt.tight_layout()
plt.show()
#plt.savefig("figs/Fig_2.b.pdf")

In [43]:
# Matched Genes